* Image denoising is to remove noise from a noisy image, so as to restore the true image
* In this notebook FER2013 dataset is used which contains approx 35 thousand images of 7 different emotions
* Image is grayscale of size 48*48

# Importing libraries

In [ ]:
from keras.datasets import fashion_mnist, mnist
import numpy as np
import matplotlib.pyplot as plt

from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Dropout
from keras.models import Model

import os,cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pylab import rcParams
rcParams['figure.figsize'] = 20, 10

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import pandas as pd# Any results you write to the current directory are saved as output.
from IPython.display import display, Image

from keras.preprocessing.image import load_img
from keras import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout, BatchNormalization
from keras.optimizers import SGD,RMSprop,adam

# Any results you write to the current directory are saved as output.
from IPython.display import display, Image

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten

## Extract data from CSV

In [ ]:
# get the data
filname = '../input/facial-expression/fer2013/fer2013.csv'

#different labels of images(not useful known about for current problem)
label_map = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

#different features names
names=['emotion','pixels','usage']

#Reading data in dataframe
df=pd.read_csv('../input/facial-expression/fer2013/fer2013.csv',names=names, na_filter=False)
im=df['pixels']
df.head(10)

## Adding labels and images(pixel values) in respective array

In [ ]:
#reading data and labels from dataset and appending in list

def getData(filname):
    # images are 48x48
    # N = 35887
    Y = []
    X = []
    first = True
    for line in open(filname):
        if first:
            first = False
        else:
            row = line.split(',')
            Y.append(int(row[0]))
            X.append([int(p) for p in row[1].split()])

    X, Y = np.array(X), np.array(Y)
    return X, Y

In [ ]:
#extracting data from dataset
X, Y = getData(filname)
num_class = len(set(Y))
#print(num_class)

## Reshaping images

In [ ]:
# keras with tensorflow backend
N, D = X.shape

#reshaping the dataset
X = X.reshape(N, 48, 48, 1)

In [ ]:
X_gaussain = X
X_poission = X
X_saltpepper = X

# Data Preprocessing

In [ ]:
#adding noise in data
noise_factor = 0.1

# Gaussian Noise

In [ ]:
#NOrmalizing the images
X_gaussain = X_gaussain.astype('float32') / 255.

#reshaping the images
X_gaussain = np.reshape(X_gaussain, (len(X_gaussain), 48, 48, 1))  # adapt this if using `channels_first` image data format

from skimage.util import random_noise

X_gaussain_noisy = X_gaussain + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_gaussain.shape)

#clipping put data near to 0--->0 aand data near to 1-->1(eg=0.3-->0 or 0.7-->1)
X_gaussain_noisy = np.clip(X_gaussain_noisy, 0., 1.)

# Poission Noise 

In [ ]:
#NOrmalizing the images
X_poission = X_poission.astype('float32') / 255.

#reshaping the images
X_poission = np.reshape(X_poission, (len(X_gaussain), 48, 48, 1))  # adapt this if using `channels_first` image data format

from skimage.util import random_noise

X_poission_noisy = X_poission + noise_factor * np.random.poisson(lam=(0,1), size=X_poission.shape) 

#clipping put data near to 0--->0 aand data near to 1-->1(eg=0.3-->0 or 0.7-->1)
X_poission_noisy = np.clip(X_poission_noisy, 0., 1.)

# Salt_Pepper Noise 

In [ ]:
#NOrmalizing the images
X_saltpepper = X_saltpepper.astype('float32') / 255.

#reshaping the images
X_saltpepper = np.reshape(X_saltpepper, (len(X_saltpepper), 48, 48, 1))  # adapt this if using `channels_first` image data format


from skimage.util import random_noise

#noisy = random_noise(img, mode="poisson")
#just change the mode pf the noise to-->'gaussain', 
X_saltpepper_noisy = random_noise(X_saltpepper, mode="s&p",clip=True, amount=noise_factor)

# Adding noise images and also creating their labels

1. gaussain-->0
1. poission-->1
1. saltpepper-->2

In [ ]:
Y_gaussain_noisy = np.zeros(X_gaussain_noisy.shape[0])
Y_poission_noisy = np.ones(X_poission_noisy.shape[0])
Y_saltpepper_noisy = np.ones(X_saltpepper_noisy.shape[0])*2

# Adding whole dataset together

In [ ]:
X_final = np.concatenate((X_gaussain_noisy, X_poission_noisy, X_saltpepper_noisy))
Y_final = np.concatenate((Y_gaussain_noisy, Y_poission_noisy, Y_saltpepper_noisy))

In [ ]:
X_final.shape

# Extracting Data and splitting train and test

In [ ]:
from keras.utils import to_categorical
Y_final = to_categorical(Y_final)

In [ ]:
#splitting data in train, test
x_train, x_test, y_train, y_test = train_test_split(X_final, Y_final, test_size=0.2, random_state=42, shuffle=True)

# Taking 500 images seperate for final testing

In [ ]:
x_test_final = x_test[-500:]
y_test_final = y_test[-500:]

x_test = x_test[:-500]
y_test = y_test[:-500]


# Visualization of 10 Data

In [ ]:
n = 10

In [ ]:
plt.figure(figsize=(48, 48))
for i in range(n):
    ax = plt.subplot(1, n, i+1)
    plt.imshow(x_train[i].reshape(48, 48))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

# Construction of Model

In [ ]:
def create_model():
    input_shape=(48,48,1)

    model = Sequential()
    model.add(Conv2D(6, (5, 5), input_shape=input_shape, padding='same', activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(16, (5, 5), padding='same', activation = 'relu'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation = 'softmax'))

    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='Adam')
    
    return model



In [ ]:
model = create_model()
model.summary()

In [ ]:
from keras import callbacks
file_path = "weights_best_.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=0, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=3)
callbacks_list = [checkpoint, early]

In [ ]:
BS = 64 
EPOCHS = 20

In [ ]:
history = model.fit(
    x_train, y_train, 
    validation_data=(x_test, y_test),
    epochs=EPOCHS, verbose=1,
    callbacks = callbacks_list 
    ,shuffle = True
    )

# Model Visualization

In [ ]:
from keras.utils import plot_model

plot_model(model, "my_first_model.png", show_shapes=True)

# Visualizing Train,Test--->Accuracy,Loss

In [ ]:
# visualizing losses and accuracy
%matplotlib inline

train_loss=history.history['loss']
val_loss=history.history['val_loss']
train_acc=history.history['accuracy']
val_acc=history.history['val_accuracy']

epochs = range(len(train_acc))

plt.plot(epochs,train_loss,'r-o', label='train_loss')
plt.plot(epochs,val_loss,'b-o', label='val_loss')
plt.title('Training loss vs Validation loss')
plt.legend()
plt.figure()

plt.plot(epochs,train_acc,'r-o', label='train_acc')
plt.plot(epochs,val_acc,'b-o', label='val_acc')
plt.title('Train Accuracy vs Validation Accuracy')
plt.legend()
plt.figure()

# Saving Model

In [ ]:
#Model Save
model.save_weights('model_weights.h5')
model.save('model_keras.h5')

# Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
results = model.predict_classes(x_test_final)

cm = confusion_matrix(np.where(y_test_final == 1)[1], results)
cm = cm.astype(np.float) / cm.sum(axis=1)[:, np.newaxis]

In [ ]:
import seaborn as sns
import pandas as pd

In [ ]:
label = ['Gaussian', 'Poission', 'Salt&Pepper']

In [ ]:
#Transform to df for easier plotting
cm_df = pd.DataFrame(cm, index = label,
                     columns = label
                    )


In [ ]:
plt.figure(figsize = (5,5))
sns.heatmap(cm_df, annot = True,cmap='Greys',cbar=False,linewidth=2)
plt.title('Noise Classify')
plt.ylabel('True class')
plt.xlabel('Prediction class')
plt.show()


# Evaluating model

In [ ]:
y_true = np.argmax(y_test_final, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_true, results)

print("Accuracy ",accuracy*100)

# Visualizing output

In [ ]:
plt.figure(figsize=(48, 48))
for i in range(3,8):
    ax = plt.subplot(1, n, i+1)
    plt.imshow(x_test_final[i].reshape(48, 48))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
y__true = np.argmax(y_test_final[3:8], axis=1)

print("True Label: ", y__true)
print("Predicted label: ", results[3:8])